In [ ]:
import pympute

print(pympute.__file__)

c:\users\benol\documents\github\python-imputation\pympute\__init__.py


In [4]:
import pandas as pd
import os
import json

#read in the data
filename = 'form_1_cleaned.csv'

filepath = os.path.join('..', 'Database', 'processed', filename)

df = pd.read_csv(filepath)


code_dict_filename = 'code_dict.json'

code_dic_path = os.path.join('..', 'Database', 'processed', code_dict_filename)

with open(code_dic_path, 'r') as file:
    code_dic = json.load(file)

C:\Users\benol\AppData\Local\Temp\ipykernel_38284\1732123383.py:10: DtypeWarning: Columns (370,371,372) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [6]:
from sys import path
path.insert(0, '../')  # Ensure pympute is importable if installed locally
import numpy as np
import pylab as plt
import pandas as pd
from copy import deepcopy
from sklearn.datasets import load_breast_cancer
from pympute import *
from pathlib import Path

# Randomly mask 20 columns
n1_miss, n2_miss = 50, 150
masked = deepcopy(df)
cols = list(df.columns)
np.random.shuffle(cols)
cols = cols[:20]
for col in cols:
    n_miss = np.random.randint(n1_miss, n2_miss)
    i_miss = np.random.randint(0, df[col].shape[0], n_miss)
    masked.loc[i_miss, col] = np.nan

# Assign imputation model for each column
model = {col: 'RF-r' for col in cols}

# Visualize missing values
masked, hold_outs = do_holdout(masked, 10)
fig, ax = plt.subplots(figsize=(25, 4))
ax.imshow(masked.T.values, interpolation='none', aspect='auto')
plt.title("Missing value heatmap")
plt.show()

# Normalize data
normin, normax = get_range(masked)
masked_n = set_range(masked, normin, normax)
data_n = set_range(df, normin, normax)

# Impute
device = 'gpu'  # change to 'cpu' if needed
if device == 'cpu':
    imp = Imputer(masked_n, model, loss_f=None, fill_method='random', save_history=True)
else:
    imp = GImputer(masked_n, model, loss_f=None, fill_method='random', save_history=True)

imp.impute(10)

# Plot loss
imp.plot_loss_frame()

TypeError: can only concatenate str (not "int") to str